# --- Day 9: Movie Theater ---


In [1]:
# --- Support Functions ---
# This section contains support functions used by the main code.

def read_input(file_path):
    with open(file_path, 'r') as file:
        return file.read().splitlines()

def get_area(a:tuple[int,int], b:tuple[int,int]) -> int:
    s1 = abs(a[0] - b[0]) + 1
    s2 = abs(a[1] - b[1]) + 1
    return s1 * s2


In [2]:
# --- Part ONE ---

input = read_input('input.txt')

tiles = [(int(x), int(y)) for x, y in [l.split(',') for l in input]]

# print (tiles)

areas = []
for i, a in enumerate(tiles):
    for b in tiles[i+1:]:
        areas.append(get_area(a, b))

#print (areas)
result = max(areas)


print("Part ONE:", result)

Part ONE: 4763040296


In [3]:
# --- Part TWO ---
input = read_input('input.txt')

result = 0


print("Part TWO:", result)

Part TWO: 0
